In [ ]:
# imports
from nibabel.testing import data_path
import numpy as np
import tensorflow as tf
from keras import applications
import nibabel as nib
import keras
import os
import pandas as pd
import re
import math

In [ ]:
# enable GPU
config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.compat.v1.Session(config=config) 
keras.backend.set_session(sess)

In [ ]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

In [ ]:
# import keras required libs.
import matplotlib.pyplot as plt
import glob
from keras.losses import categorical_crossentropy
from keras.models import Sequential, Model 
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras import layers
from sklearn.model_selection import train_test_split

In [ ]:
# bringing in dataframes
df = pd.read_csv('OASIS4_data_clinical.csv')
neuro = pd.read_csv('OASIS4_data_Neuropsychometric.csv')

In [ ]:
neuro.head()

In [ ]:
neuro = neuro[['oasis_id','short_blessed']]
for i in range(len(neuro['oasis_id'])):
    if isinstance(neuro['short_blessed'][i], str):
        if (neuro['short_blessed'][i].isnumeric()):
            neuro.at[i,'short_blessed']=int(neuro['short_blessed'][i])
        else:
            neuro = neuro.drop(labels=i, axis=0)
    elif math.isnan(neuro['short_blessed'][i]):
        neuro = neuro.drop(labels=i, axis=0)
neuro = neuro.reset_index(drop=True)
for i in range(len(neuro['oasis_id'])-1):
    if neuro['oasis_id'][i] == neuro['oasis_id'][i+1]:
        neuro = neuro.drop(labels=i, axis=0)
neuro = neuro.reset_index(drop=True)
#for i in range(len(neuro['oasis_id'])):
#    if 

In [ ]:
# Regex
pattern = r"\d{5}"
neuro.head()

In [ ]:
print(type(neuro['oasis_id']))

In [ ]:
# fetch the files we need.
filename_list = []
for name in glob.glob('./NIIs only TEST/*.nii.gz'):
    filename_list.append(name)
# Dealing with missing images/ clinical entries
print(len(filename_list))
x=0
while x < len(filename_list):
    if "run-01" in filename_list[x] or "run-03" in filename_list[x] or "run-04" in filename_list[x] or "run-05" in filename_list[x]:
        os.remove(filename_list[x])
        filename_list.pop(x)
    x+=1
print(x)

In [ ]:
x=0
while x < len(filename_list):
    match = re.search(pattern, filename_list[x])
    file_num = match.group(0)
    match = re.search(pattern, df['oasis_id'][x])
    clinical_num = match.group(0)
    if (int(file_num) < int(clinical_num)):
        os.remove(filename_list[x])
        filename_list.pop(x)
    elif (int(file_num) > int(clinical_num)):
        df = df.drop(labels=x, axis=0)
        df = df.reset_index(drop=True)
    x+=1
print(x)

In [ ]:
for x in range (len(filename_list)):
    if df['oasis_id'][x] in filename_list[x]:
        continue 
    else:
        print(x)

In [ ]:
neuro_list=[]
for x in range (len(neuro['oasis_id'])):
    neuro_list.append(neuro['oasis_id'][x])

print(len(neuro_list))


df_list=[]
for x in range (len(df['oasis_id'])):
    df_list.append(df['oasis_id'][x])

print(len(df_list))

In [ ]:
filename_drops = []
for x in range (len(neuro['oasis_id'])):
    if neuro_list[x] not in df_list:
        filename_drops.append(neuro['oasis_id'][x])         
        neuro = neuro.drop(labels=x, axis=0)


for x in range (len(df['oasis_id'])):
    if df_list[x] not in neuro_list:
        filename_drops.append(df['oasis_id'][x])        
        df = df.drop(labels=x, axis=0)

neuro = neuro.reset_index(drop=True)
df = df.reset_index(drop=True)

print(len(neuro['oasis_id']))
print(len(neuro['oasis_id']))

In [ ]:
for x in range (len(df['oasis_id'])):
    if df['oasis_id'][x] in neuro['oasis_id'][x]:
        continue 
    else:
        print(x)

In [ ]:
print(len(filename_list))
for i in range (len(filename_list)):
    match = re.search(pattern, filename_list[i])
    file_num = 'OAS'+str(match.group(0))
    if file_num in filename_drops:
        filename_list.pop(i)

print(len(filename_list))


In [ ]:
# Cutting the tail values we don't need and making a
# column of the corresponding images.

df['image_path'] = filename_list
df.shape[0]
df.tail()
df.to_csv('cleaned.csv')
print(len(df['oasis_id']))
print(len(neuro['oasis_id']))
df.head()

In [ ]:
dx_keep = [0,1,2]
for i in range(df.shape[0]-1):
    if df['final_dx_categorized'][i] not in dx_keep:
        df = df.drop(labels=i, axis=0)
        neuro = neuro.drop(labels=i, axis=0)
df = df.reset_index(drop=True)
neuro = neuro.reset_index(drop=True)
print(neuro.shape)
print(df.shape)

In [ ]:
df = df[['oasis_id','age','declong','final_dx_categorized','image_path']]
neuro = neuro[['short_blessed']]
clinical = pd.concat([df, neuro], axis=1, join='inner')
display(df)
print(type(df['declong'][0]))
display(neuro)

In [ ]:
# Label the images
cn_x = []
cn_y = []
ad_x = []
ad_y = []
mci_x = []
mci_y = []
total_data_x = []
total_data_y = []
chosen_images = []
for i in range(df.shape[0]):
    if df['final_dx_categorized'][i] == 0:
        newimg = nib.load(df['image_path'][i])
        data = newimg.get_fdata()
        if data.shape==(320,320,28):
            clinical_values = [df['age'][i], df['declong'][i], neuro['short_blessed'][i]]
            reshaped_values = np.array(clinical_values).reshape((1, 1, 3))
            resized_array = np.resize(reshaped_values, (320, 320, 3))
            result = np.concatenate((data, resized_array), axis=2)
            cn_x.append(result)
            cn_y.append(0)
            chosen_images.append(df['oasis_id'][i])
    elif df['final_dx_categorized'][i] == 1:
        newimg = nib.load(df['image_path'][i])
        data = newimg.get_fdata()
        if data.shape==(320,320,28):
            clinical_values = [df['age'][i], df['declong'][i], neuro['short_blessed'][i]]
            reshaped_values = np.array(clinical_values).reshape((1, 1, 3))
            resized_array = np.resize(reshaped_values, (320, 320, 3))
            result = np.concatenate((data, resized_array), axis=2)
            ad_x.append(result)
            ad_y.append(1)
            chosen_images.append(df['oasis_id'][i])
    elif df['final_dx_categorized'][i] == 2:
        newimg = nib.load(df['image_path'][i])
        data = newimg.get_fdata()
        if data.shape==(320,320,28):
            clinical_values = [df['age'][i], df['declong'][i], neuro['short_blessed'][i]]
            reshaped_values = np.array(clinical_values).reshape((1, 1, 3))
            resized_array = np.resize(reshaped_values, (320, 320, 3))
            result = np.concatenate((data, resized_array), axis=2)
            mci_x.append(result)
            mci_y.append(2)
            chosen_images.append(df['oasis_id'][i])
for i in range(len(cn_x)):
    total_data_x.append(cn_x[i])
    total_data_y.append(cn_y[i])
for i in range(len(ad_x)):
    total_data_x.append(ad_x[i])
    total_data_y.append(ad_y[i])
for i in range(len(mci_x)):
    total_data_x.append(mci_x[i])
    total_data_y.append(mci_y[i])
# Cut out from neuro set:

for i in range(len(total_data_x)):
    print(total_data_x[i].shape)
print(len(total_data_x))

In [ ]:
newdata=np.array(total_data_x, dtype=object)
print(type(newdata))
print(newdata.shape)
data_length = len(total_data_x)
zeros_array = np.zeros((data_length, 320, 320, 1))
newdata = np.concatenate((newdata, zeros_array), axis=3)
print(newdata.shape)
maindata=np.stack([newdata]*3, axis=-1)
maindata=maindata.reshape(data_length*320,320,32,3)

In [ ]:
new_y=[]
for i in range (len(total_data_x)):
    for j in range(320):
        new_y.append(total_data_y[i])
new_y=np.array(new_y)

In [ ]:
maindata.shape,new_y.shape

In [ ]:
# splitting data for train and test 70:30.
X_train, X_test, y_train, y_test = train_test_split(maindata, new_y, test_size=0.33)

X_train=np.array(X_train)
y_train=np.array(y_train)
x_train = X_train.reshape(X_train.shape[0], 320, 32, 3)

input_shape = (320,32,3)
x_train = x_train.astype('float32')
y_train = keras.utils.to_categorical(y_train)
y_old_test=y_test
y_test = keras.utils.to_categorical(y_test)


In [ ]:
# Using ResNet50

model = applications.ResNet50(weights = "imagenet", include_top=False, input_shape=input_shape)


# Freeze the layers which you don't want to train. Here I am not freezing any layer.
for layer in model.layers:
    layer.trainable = True
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation="softmax")(x)
model= Model(model.input, predictions)

In [ ]:
# Compile and Train:
model.compile(keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
val=model.fit(x=x_train,y=y_train,batch_size=64, epochs=10,validation_split=0.1)

In [ ]:
X_test=np.array(X_test)
x_test = X_test.reshape(X_test.shape[0],320,32,3)
x_test = x_test.astype('float32')
    # Normalizing the RGB codes by dividing it to the max RGB value.
print('Number of images in x_test', x_test.shape[0])
model.evaluate(x_test,y_test)


In [ ]:
plt.plot(val.history['accuracy'], color='g')
plt.plot(val.history['val_accuracy'], color='b')
plt.legend(['Training accuracy' ,'Validation accuracy'])
plt.title("Train Vs Validation Accuracy")
plt.xlabel("# of Epoch")
plt.ylabel("Accuracy")
plt.ylim(0.40,1)
plt.savefig('TrainVSvalidation.png')